In [11]:
import boto3
import sagemaker

boto_session = boto3.Session(region_name="us-east-1")
sagemaker_client = boto_session.client("sagemaker")
runtime_client = boto_session.client("sagemaker-runtime")
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=runtime_client,
    # default_bucket=default_bucket,
)

role = sagemaker.session.get_execution_role(sagemaker_session)
bucket = sagemaker_session.default_bucket()
prefix = "Scikit-LinearLearner-pipeline-abalone-example"

In [16]:
bucket

'sagemaker-us-east-1-926521026587'

# Preprocessing data and training the model <a class="anchor" id="training"></a>
## Downloading dataset <a class="anchor" id="download_data"></a>
SageMaker team has downloaded the dataset from UCI and uploaded to one of the S3 buckets in our account.

In [15]:
!wget --directory-prefix=./abalone_data https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv

--2021-06-17 17:20:27--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/data/abalone/abalone.csv
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.241.8
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.241.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [binary/octet-stream]
Saving to: ‘./abalone_data/abalone.csv’

100%[======================================>] 191,873      780KB/s   in 0.2s   

2021-06-17 17:20:27 (780 KB/s) - ‘./abalone_data/abalone.csv’ saved [191873/191873]



## Upload the data for training <a class="anchor" id="upload_data"></a>

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. We can use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [17]:
WORK_DIRECTORY = "abalone_data"

train_input = sagemaker_session.upload_data(
    path=f"{WORK_DIRECTORY}/abalone.csv",
    bucket=bucket,
    key_prefix=f"{prefix}/train",
)

In [19]:
train_input 

's3://sagemaker-us-east-1-926521026587/Scikit-LinearLearner-pipeline-abalone-example/train/abalone.csv'

## Create SageMaker Scikit Estimator <a class="anchor" id="create_sklearn_estimator"></a>

To run our Scikit-learn training script on SageMaker, we construct a `sagemaker.sklearn.estimator.sklearn` estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __framework_version__: Scikit-learn version you want to use for executing your model training code.
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.

To see the code for the SKLearn Estimator, see here: https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/sklearn

In [18]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = "sklearn_abalone_featurizer.py"

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

In [21]:
sklearn_preprocessor.fit({"train": train_input})

2021-06-17 17:37:19 Starting - Starting the training job...
2021-06-17 17:37:43 Starting - Launching requested ML instancesProfilerReport-1623951439: InProgress
......
2021-06-17 17:38:43 Starting - Preparing the instances for training.........
2021-06-17 17:40:20 Downloading - Downloading input data
2021-06-17 17:40:20 Training - Downloading the training image...
2021-06-17 17:40:49 Uploading - Uploading generated training model.2021-06-17 17:40:44,934 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-06-17 17:40:44,936 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-17 17:40:44,945 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-06-17 17:40:45,321 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-17 17:40:45,334 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-17 17:40:45,347 sagemaker-

## Batch transform our training data <a class="anchor" id="preprocess_train_data"></a>
Now that our proprocessor is properly fitted, let's go ahead and preprocess our training data. Let's use batch transform to directly preprocess the raw data and store right back into s3.

In [22]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, instance_type="ml.m5.xlarge", assemble_with="Line", accept="text/csv"
)

In [23]:
# Preprocess training input
transformer.transform(train_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

.............................2021-06-17 17:48:49,234 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-06-17 17:48:49,236 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-06-17 17:48:49,237 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
2021-06-17 17:48:49,234 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-06-17 17:48:49,236 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-06-17 17:48:49,237 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client

In [24]:
preprocessed_train

's3://sagemaker-us-east-1-926521026587/sagemaker-scikit-learn-2021-06-17-17-44-16-608'

## Fit a LinearLearner Model with the preprocessed data <a class="anchor" id="training_model"></a>
Let's take the preprocessed training data and fit a LinearLearner Model. Sagemaker provides prebuilt algorithm containers that can be used with the Python SDK. The previous Scikit-learn job preprocessed the raw Titanic dataset into labeled, useable data that we can now use to fit a binary classifier Linear Learner model.

For more on Linear Learner see: https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html

In [27]:
from sagemaker.image_uris import retrieve

ll_image = retrieve("linear-learner", boto_session.region_name)

In [28]:
s3_ll_output_key_prefix = "ll_training_output"
s3_ll_output_location = "s3://{}/{}/{}/{}".format(
    bucket, prefix, s3_ll_output_key_prefix, "ll_model"
)

ll_estimator = sagemaker.estimator.Estimator(
    ll_image,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge",
    volume_size=20,
    max_run=3600,
    input_mode="File",
    output_path=s3_ll_output_location,
    sagemaker_session=sagemaker_session,
)

ll_estimator.set_hyperparameters(feature_dim=10, predictor_type="regressor", mini_batch_size=32)

ll_train_data = sagemaker.inputs.TrainingInput(
    preprocessed_train,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)

data_channels = {"train": ll_train_data}
ll_estimator.fit(inputs=data_channels, logs=True)

293309674813197}}}

#metrics {"StartTime": 1623952654.7730486, "EndTime": 1623952654.7730584, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 21}, "Metrics": {"train_mse_objective": {"sum": 0.944401360847629, "count": 1, "min": 0.944401360847629, "max": 0.944401360847629}}}

#metrics {"StartTime": 1623952654.7730958, "EndTime": 1623952654.7731044, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 22}, "Metrics": {"train_mse_objective": {"sum": 0.8293406346669564, "count": 1, "min": 0.8293406346669564, "max": 0.8293406346669564}}}

#metrics {"StartTime": 1623952654.7731347, "EndTime": 1623952654.7731426, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 23}, "Metrics": {"train_mse_objective": {"sum": 0.944387152332526, "count": 1, "min": 0.944387152332526, "max": 0.944387152332526}}}

#metrics {"StartTime

# Serial Inference Pipeline with Scikit preprocessor and Linear Learner <a class="anchor" id="serial_inference"></a>


## Set up the inference pipeline <a class="anchor" id="pipeline_setup"></a>
Setting up a Machine Learning pipeline can be done with the Pipeline Model. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model. Deploying the model follows the same ```deploy``` pattern in the SDK.

In [31]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
linear_learner_model = ll_estimator.create_model()

model_name = "inference-pipeline-" + timestamp_prefix
endpoint_name = "inference-pipeline-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[scikit_learn_inferencee_model, linear_learner_model]
)

In [33]:
sm_model.deploy(initial_instance_count=1, instance_type="ml.c4.xlarge", endpoint_name=endpoint_name)

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

## Make a request to our pipeline endpoint <a class="anchor" id="pipeline_inference_request"></a>

Here we just grab the first line from the test data (you'll notice that the inference python script is very particular about the ordering of the inference request data). The ```ContentType``` field configures the first container, while the ```Accept``` field configures the last container. You can also specify each container's ```Accept``` and ```ContentType``` values using environment variables.

We make our request with the payload in ```'text/csv'``` format, since that is what our script currently supports. If other formats need to be supported, this would have to be added to the ```output_fn()``` method in our entry point. Note that we set the ```Accept``` to ```application/json```, since Linear Learner does not support ```text/csv``` ```Accept```. The prediction output in this case is trying to guess the number of rings the abalone specimen would have given its other physical features; the actual number of rings is 10.

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

payload = "M, 0.44, 0.365, 0.125, 0.516, 0.2155, 0.114, 0.155"
actual_rings = 10
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session, serializer=CSVSerializer()
)

print(predictor.predict(payload))

## Delete Endpoint <a class="anchor" id="delete_endpoint"></a>
Once we are finished with the endpoint, we clean up the resources!

In [ ]:
sm_client = sagemaker_session.boto_session.client("sagemaker")
sm_client.delete_endpoint(EndpointName=endpoint_name)